In [3]:
!pip install openprompt

import pandas as pd
from openprompt.data_utils import InputExample
import torch
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score,f1_score,classification_report
from sklearn.model_selection import train_test_split
from openprompt.config import get_config, save_config_to_yaml
from yacs.config import CfgNode
from tqdm.contrib import tenumerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.4 MB/s eta 0:00:0000:01
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99


/opt/conda/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5

In [4]:
path=r"/kaggle/input/rs-2012-03"
df=pd.read_csv(os.path.join(path,"RS_2012-03_violations2.csv"))

In [5]:
classes = [ # There are two classes in Sentiment Analysis, one for negative and one for positive
    "racial_slur",
    "self_harm",
    "Homophobia",
    "Incivility",
    "harrassment",
    "No Violations"
]

encoder = LabelEncoder()
df["label"]=encoder.fit_transform(df["violation"])

le_name_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

print(le_name_mapping)



{'Homophobia': 0, 'Incivility': 1, 'No Violations': 2, 'harrassment': 3, 'racial_slur': 4, 'self_harm': 5}


In [6]:
from openprompt.plms import load_plm
from openprompt.lm_bff_trainer import LMBFFClassificationRunner,ClassificationRunner
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from openprompt.prompts import ManualTemplate

promptTemplate = ManualTemplate(
    text = '{"placeholder":"text_a"} was the context for {"placeholder":"text_b"}, Which violation is it? {"mask"}' ,
    tokenizer = tokenizer,
)

bad_words = {
    "racial_slur": ["nigga","nigger","uncle tom","negro","niggerhead","house slave","monkeyboy"],
    "self_harm": ["kill yourself", "commit suicide"],
    "Homophobia":["lesbo","faggot","fag",'cocksucker'],
    "Incivility":["dickhead","twat","cunt","whore","retard","bitch","asshole","dimwit","bullshit","fuck u","fuck you","motherfuck"],
    "harrassment":["penis","pussy","dick","tits","moron","suck dick","sexy bitch"]
}

In [8]:
bad_set=set()
for k,v in bad_words.items():
    for i in bad_words[k]: 
        bad_set.add(i)
all_set=set()
for i in df["context"]:
   for k in i.split(' '):
       all_set.add(k.lower())
    
no_vio = all_set.difference(bad_set)  
no_vio=list(no_vio)
no_vio=no_vio[0:len(no_vio):5000]

    
 
from openprompt.prompts import ManualVerbalizer
promptVerbalizer = ManualVerbalizer(
    classes = classes,
    label_words = {
        "racial_slur": ["nigga","nigger","uncle tom","negro","niggerhead","house slave","monkeyboy"],
        "self_harm": ["kill yourself", "commit suicide"],
        "Homophobia":["lesbo","faggot","fag",'cocksucker'],
        "Incivility":["dickhead","twat","cunt","whore","retard","bitch","asshole","dimwit","bullshit","fuck u","fuck you","motherfuck"],
        "harrassment":["penis","pussy","dick","tits","moron","suck dick","sexy bitch"],
        "No Violations": no_vio
    },
    tokenizer = tokenizer,
)

In [9]:
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
    freeze_plm=False,
    plm_eval_mode=False
)


In [10]:
data_train , data_val = train_test_split(df,train_size=0.75,random_state=2018,stratify=df['label'])
#data_train,data_val = train_test_split(data_train_val,train_size=0.25,random_state=2018,stratify=data_train_val['label'])

def InputExampleConverter(df): 
    dataset=[]
    count=0
    violation_sentence = df["sentence"]
    label=df["label"]
    context=df["context"]
    print("Creating Dataset")     
    for i in violation_sentence: 
        dataset.append(InputExample(guid=count,text_a=context[count],text_b=i,label=label[count]))
        count=count+1
    return(dataset)
data_train=InputExampleConverter(data_train.reset_index())
data_val=InputExampleConverter(data_val.reset_index())
#data_test=InputExampleConverter(data_test.reset_index())    

        


from openprompt import PromptDataLoader
data_loader_train = PromptDataLoader(
    dataset = data_train,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    shuffle=True
)

data_loader_val = PromptDataLoader(
    dataset = data_val,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    shuffle=True
)
'''
data_loader_test = PromptDataLoader(
    dataset = data_test,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    shuffle=True
)
'''

Creating Dataset
Creating Dataset


tokenizing: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1524 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 45762it [20:20, 37.50it/s]
tokenizing: 15254it [06:53, 36.87it/s]


'\ndata_loader_test = PromptDataLoader(\n    dataset = data_test,\n    tokenizer = tokenizer,\n    template = promptTemplate,\n    tokenizer_wrapper_class=WrapperClass,\n    shuffle=True\n)\n'

In [11]:
predictions=[]

use_cuda = True
if use_cuda:
    promptModel=  promptModel.cuda()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
optimizer = AdamW(promptModel.parameters(), lr=1e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

for epoch in range(10):
    tot_loss = 0
    prev_score=0
    val_score=[]
    test_score=[]
    for step, inputs in tenumerate(data_loader_train):
        if use_cuda:
            inputs = inputs.to(device) 
        #print(step)   
        logits = promptModel(inputs)
        logits=logits.to(device)
        labels = inputs['label']
        labels=labels.to(device)
        loss = loss_func(logits,labels.type(torch.LongTensor).to(device))
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step %5000 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)
            
            try: 
                torch.save({

            'epoch': epoch,

            'model_state_dict': promptModel.state_dict(),

            'optimizer_state_dict': optimizer.state_dict(),

            'loss': loss,

            },os.path.join(path,"violation_type_model_chkpt.pt"))
                
            except:
                continue
    allpreds = []
    alllabels = []
    print("getting val score after epoch "+ str(epoch))
    for step, inputs in tenumerate(data_loader_val):
        if use_cuda:
            inputs = inputs.cuda()
        logits = promptModel(inputs)
        labels = inputs['label']
        alllabels.extend(labels.cpu().tolist())
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

    print(f1_score(alllabels,allpreds,average="micro" ))
    print(classification_report(alllabels,allpreds))  
    
    

  0%|          | 0/45762 [00:00<?, ?it/s]

Epoch 0, average loss: 1.7903161644935608
Epoch 0, average loss: 1.0216323076659384
Epoch 0, average loss: 0.9638755424967996
Epoch 0, average loss: 0.9330157394077179
Epoch 0, average loss: 0.9132328703087899
Epoch 0, average loss: 0.8974678920088283
Epoch 0, average loss: 0.8870737991765966
Epoch 0, average loss: 0.8765657370096896
Epoch 0, average loss: 0.8680584327154751
Epoch 0, average loss: 0.8632002104879795
getting val score after epoch 0


  0%|          | 0/15254 [00:00<?, ?it/s]

0.6718893405008523
              precision    recall  f1-score   support

           0       0.90      0.06      0.12       280
           1       0.66      0.40      0.50      4311
           2       0.67      0.92      0.78      8460
           3       0.75      0.33      0.46      1954
           4       0.70      0.20      0.31       194
           5       1.00      0.09      0.17        55

    accuracy                           0.67     15254
   macro avg       0.78      0.34      0.39     15254
weighted avg       0.68      0.67      0.64     15254



  0%|          | 0/45762 [00:00<?, ?it/s]

...

Epoch 5, average loss: 0.2584132557094563
Epoch 5, average loss: 0.49438531662708
Epoch 5, average loss: 0.5055921148100418
Epoch 5, average loss: 0.511271701747852
Epoch 5, average loss: 0.5203532196134836
Epoch 5, average loss: 0.5294188705465258
Epoch 5, average loss: 0.5338809722918668
Epoch 5, average loss: 0.538984329722015
Epoch 5, average loss: 0.5438432304694156
Epoch 5, average loss: 0.5508875655775383
getting val score after epoch 5


  0%|          | 0/15254 [00:00<?, ?it/s]

0.6731349154320179
              precision    recall  f1-score   support

           0       0.64      0.20      0.31       280
           1       0.60      0.48      0.53      4311
           2       0.73      0.84      0.78      8460
           3       0.55      0.48      0.51      1954
           4       0.53      0.43      0.47       194
           5       0.41      0.31      0.35        55

    accuracy                           0.67     15254
   macro avg       0.58      0.46      0.49     15254
weighted avg       0.66      0.67      0.66     15254



  0%|          | 0/45762 [00:00<?, ?it/s]

Epoch 6, average loss: 0.7851646244525909
Epoch 6, average loss: 0.46561491552532924
Epoch 6, average loss: 0.4732228925932715
Epoch 6, average loss: 0.4820455273505272
Epoch 6, average loss: 0.4861551356448
Epoch 6, average loss: 0.49380886002308355
Epoch 6, average loss: 0.5011107444223148
Epoch 6, average loss: 0.5092869174591674
Epoch 6, average loss: 0.5126835792966533


In [ ]:
torch.save(promptModel,os.path.join(path,"violation_detector.pt"))